# Intro

the dataset that will be used in this notebook is the [**Mapping 2019-nCoV**](https://systems.jhu.edu/research/public-health/ncov/) which contains several datasources (including the **WHO** and the **ECDC**) and it can be found [here](https://github.com/CSSEGISandData/COVID-19)

Particually, we will use the **CSSE COVID-19 Dataset** folder, which contains two *types* of data:

- Daily reports (csse_covid_19_daily_reports): <br>
    this folder contains daily aggregated case reports, wich contains information such the number of confirmed cases, the number of deaths, and the number of recovered cases.
    each report contains accumulated till the date of the report.
    we can use the last report to show statistic measures such as total number of cases for each country, most affected countries, etc ..., but we can perform time analysis on the data.
- Time series summary (csse_covid_19_time_series):

# Import Libraries and Read Data

In [ ]:
import glob
import pandas as pd
import numpy as np

import plotly
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px

import plotly.graph_objects as go
from plotly.graph_objects import Bar
from plotly.subplots import make_subplots

init_notebook_mode(connected=True)
plotly.offline.init_notebook_mode(connected=True)

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

sns.set()

In [ ]:
def get_last_report_date(reports_path):
    
    # this is a buggy code, that just works!
    # this function is used to get the date of the last report data in csse_covid_19_daily_reports folder
    
    files_names_list = glob.glob(f"{reports_path}/*.csv")
    
    files_names_list = sorted(files_names_list)
    
    last_report_name = files_names_list[-1].split("/")[-1]
    
    last_report_date = last_report_name.split(".")[0]
    
    return last_report_date

In [ ]:
last_report_date = get_last_report_date("COVID-19/csse_covid_19_data/csse_covid_19_daily_reports")

In [ ]:
last_report_date

In [ ]:
last_report_path = f'COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/{last_report_date}.csv'
world_pop_path = 'other-data/World Population Data/total_population_clean.csv'
pop_ages_1_path = 'other-data/World Population Data/population_ages_0-14_clean.csv'
pop_ages_2_path = 'other-data/World Population Data/population_ages_15-64_clean.csv'
pop_ages_3_path = 'other-data/World Population Data/population_ages_65_clean.csv'

In [ ]:
df = pd.read_csv(last_report_path)
world_pop_df = pd.read_csv(world_pop_path)
pop_ages_group_1_df = pd.read_csv(pop_ages_1_path)
pop_ages_group_2_df = pd.read_csv(pop_ages_2_path)
pop_ages_group_3_df = pd.read_csv(pop_ages_3_path)

# Helper functions

In [ ]:
def save_fig_as_div(fig_obj, file_name):
    with open(f'charts/{file_name}', 'w') as fig_file:
        fig_div_string = plotly.offline.plot(figure_or_data=fig_obj, output_type='div',
                                             include_plotlyjs='cdn')
        fig_file.write(fig_div_string)

In [ ]:
def get_n_color_palette(palette_name, n_colors, as_hex=False):
    palette = sns.color_palette(palette=palette_name, n_colors=n_colors)
    if as_hex:
        palette = palette.as_hex()
    palette.reverse()
    return palette

In [ ]:
def get_most_n_affected_countries(df, n):
    most_affected_countries_names = df['Country'][:n].values
    
    most_affected_df = df[df['Country'].isin(most_affected_countries_names)]
    
    return most_affected_df

In [ ]:
def get_countries_fatalities(df):
    countries_fatalities_df = pd.DataFrame(data=df[['Country', 'Confirmed', 'Deaths']])

    countries_fatalities_df['Fatality rate'] = (countries_fatalities_df['Deaths'] * 100.0) / countries_fatalities_df['Confirmed']

    countries_fatalities_df = countries_fatalities_df.round({'Fatality rate': 2})

    countries_fatalities_df.sort_values(by='Confirmed', ascending=False, inplace=True)
    
    return countries_fatalities_df

# Get to know the data

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
len(df['Country_Region'].unique())

Field description
- Province/State: China - province name; US/Canada/Australia/ - city name, state/province name; Others - name of the event (e.g., "Diamond Princess" cruise ship); other countries - blank.
- Country/Region: country/region name conforming to WHO (will be updated).
- Last Update: MM/DD/YYYY HH:mm (24 hour format, in UTC).
- Confirmed: the number of confirmed cases.
- Deaths: the number of deaths.
- Recovered: the number of recovered cases.
- Active: Active cases = total confirmed - total recovered - total deaths.

## Populations Ages groups

In [ ]:
pop_ages_group_1_df.head()

In [ ]:
pop_ages_group_2_df.head()

In [ ]:
pop_ages_group_3_df.head()

## Countries age structure statistics:

This data contains the percentages of age groups of population in each country, the age groups are:

- Ages 0 to 14 years: children and adolescents.
- Ages 15 to 64 years: working population or population in education
- Over the age of 65: retirees; elderly

will use this data later to study the relation between death rate and percentage of elderly.

In [ ]:
countries_ages_groups_df = pop_ages_group_1_df.merge(pop_ages_group_2_df, left_on='Country',
                                                     right_on='Country', how='inner',
                                                     suffixes=('_group_1', '_group_2'))\
    .merge(pop_ages_group_3_df, left_on='Country', right_on='Country', how='inner')\
    .drop(columns=['Country Code_group_1', 'Country Code_group_2', 'Country Code'])\
    .rename(columns={
        'percentage_group_1': '0 to 14 years',
        'percentage_group_2': '15 to 64 years',
        'percentage': 'over 65 years'
    })

In [ ]:
countries_ages_groups_df.head()

# Data Preprocessing

In [ ]:
df.head()

## 1 - Drop irrelevant columns:

`FIPS`: the **FIPS** represents the Federal Information Processing Standard state code, and it's related only to the `U.S.` states, we won't need it now (unless we want to draw a map specific to the `U.S.`)

`Admin2`: it's not clear what does this column means, will be dropped :)

`Province_State`: the city on which the case was recorded, but not all cases are associated with the city, so we will drop it.

In [ ]:
df['Province_State'].isna().sum()

`Last_update`: the date on which this row was last updated, irrelvant to our analysis.

`Lat` and `Long_` columns specify the coordinates of the case, we won't be needing this inofrmation right now.

`Combined_Key`: this column is the concatenation of `Admin2`, `Province_State`, and ` 	Country_Region`, irrelvant too.

In [ ]:
def drop_irrelevant_columns(df, irrelevant_columns):
    new_df = df.drop(columns=irrelevant_columns)
    return new_df

In [ ]:
irrelevant_columns = ['FIPS', 'Admin2', 'Province_State', 'Last_Update',
                      'Lat', 'Long_', 'Combined_Key']

In [ ]:
df = drop_irrelevant_columns(df, irrelevant_columns)

In [ ]:
df.head()

## 2 - Normalize columns' names:

rename column `Country_Region` to `Country`, for easier manipulation

In [ ]:
def rename_columns(df, columns_mapping):
    new_df = df.rename(columns=columns_mapping)
    return new_df

In [ ]:
columns_mapping = {
    'Country_Region': 'Country'
}

In [ ]:
df = rename_columns(df, columns_mapping)

In [ ]:
df.head()

## 3 - Aggregate data by countries:
aggregate the number of confirmed cases, number of deaths, number of reovered cases, and number of active cases for each country.

In [ ]:
def apply_aggregation(df, group_by_column, sort_by_column):
    aggregate_df = df.groupby(by=group_by_column)\
                        .sum()\
                        .reset_index()\
                        .sort_values(by=sort_by_column, ascending=False)
    return aggregate_df

In [ ]:
aggregated_df = apply_aggregation(df, group_by_column='Country', sort_by_column='Deaths')

In [ ]:
aggregated_df.head(10)

the number of `Confirmed` cases in each country should be equal to the sum of: `Deaths`, `Recovered`, and `Active`

In [ ]:
aggregated_df['Confirmed'].eq(aggregated_df['Deaths'] + aggregated_df['Recovered'] + aggregated_df['Active']).sum()

In [ ]:
aggregated_df.shape

## 4 - Focus on *Outbreaks*:
We will focus in this notebook only on countries with high **death tolls**, countires that are *nearly* affected by the pandemic will be discarded.

In [ ]:
most_n_countries = 15

In [ ]:
most_affected_countries_names = aggregated_df['Country'][:most_n_countries].values

In [ ]:
print(most_affected_countries_names)

In [ ]:
most_affected_df = aggregated_df[aggregated_df['Country'].isin(most_affected_countries_names)]

In [ ]:
most_affected_df

# Data Visualization

In [ ]:
reds = get_n_color_palette('Reds', n_colors=most_n_countries, as_hex=True)

## Overall Statistics

In [ ]:
overall_stats_df = pd.DataFrame(df[['Deaths', 'Recovered', 'Active']].sum(), columns=['total'])\
                                .reset_index()\
                                .rename(columns={
                                        'index': 'measure'
                                })

In [ ]:
overall_stats_df.head()

In [ ]:
fig = px.pie(data_frame=overall_stats_df, values='total', names='measure',
             color='measure', color_discrete_map={
                 'Active': 'orange',
                 'Recovered': 'green',
                 'Deaths': 'red'
             })

fig.update_layout(autosize=False,
                  width=800, height=600,
                  title_text='<i><b>Overall statistics</b></i>',
                  xaxis_title="Measures", yaxis_title="Class",
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  paper_bgcolor= 'rgba(0, 0, 0, 0)',
                  font={
                      'family': "Courier New, monospace",
                      'size': 14,
                      'color': "#eaeaea"
                  }
                 )

fig.show()

In [ ]:
save_fig_as_div(fig, 'overall_stats_pie_chart.html')

## Pie Chart

In [ ]:
fig = px.pie(most_affected_df, names='Country', values='Deaths',
             color='Deaths', color_discrete_sequence=reds)

fig.update_layout(autosize=False,
                  width=800, height=700,
                  title_text='<i><b>Most affected 15 countries by deaths</b></i>',
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  paper_bgcolor= 'rgba(0, 0, 0, 0)',
                  font={
                      'family': "Courier New, monospace",
                      'size': 14,
                      'color': "#eaeaea"
                  }
                 )

fig.show()

In [ ]:
save_fig_as_div(fig_obj=fig, file_name="most_affected_countries_pie_chart.html")

## Treemap Chart

In [ ]:
country_continent_df = pd.read_csv("other-data/Countries-Continents.csv")

In [ ]:
country_continent_df.head()

In [ ]:
most_n = 30

In [ ]:
treemap_df = get_most_n_affected_countries(aggregated_df, 30).merge(right=country_continent_df,
                                left_on='Country',
                                right_on='Country',
                                how='inner')

In [ ]:
treemap_df.head()

In [ ]:
treemap_df["world"] = "world" # in order to have a single root node

In [ ]:
fig = px.treemap(treemap_df,
                 path=['world', 'Continent', 'Country'], values='Deaths',
                 color='Deaths', color_continuous_scale=['orange', 'red'])

fig.update_layout(autosize=False,
                  width=800, height=600,
                  title_text='<i><b>Treemap of Country-Continent</b></i>',
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  paper_bgcolor= 'rgba(0, 0, 0, 0)',
                  font={
                      'family': "Courier New, monospace",
                      'size': 14,
                      'color': "#eaeaea"
                  }
                 )

fig.show()

In [ ]:
save_fig_as_div(fig_obj=fig, file_name='most_affected_countries_treemap_chart.html')

## Stacked Bar Chart:

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='Deaths', x=most_affected_df['Country'], y=most_affected_df['Deaths'], marker_color='red'),
    go.Bar(name='Recovered', x=most_affected_df['Country'], y=most_affected_df['Recovered'], marker_color='green'),
    go.Bar(name='Active', x=most_affected_df['Country'], y=most_affected_df['Active'], marker_color='orange'),
])

fig.update_layout(barmode='stack', autosize=False,
                  width=800, height=600,
                  title_text='<i><b>Most affected 15 countries by deaths</b></i>',
                  xaxis_title="Country", yaxis_title="Cases",
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  paper_bgcolor= 'rgba(0, 0, 0, 0)',
                  font={
                      'family': "Courier New, monospace",
                      'size': 14,
                      'color': "#eaeaea"
                  }
                 )

fig.show()

In [ ]:
save_fig_as_div(fig_obj=fig, file_name='most_affected_countries_stacked_bar_chart.html')

## Countries fatalities rate:

In [ ]:
countries_fatalities_df = pd.DataFrame(data=most_affected_df[['Country', 'Confirmed', 'Deaths']])

In [ ]:
countries_fatalities_df.head()

let's compute the fatality rate:

The fatality rate of a country, is: (deaths * 100) / confirmed

In [ ]:
countries_fatalities_df['Fatality rate'] = (countries_fatalities_df['Deaths'] * 100) / countries_fatalities_df['Confirmed']

In [ ]:
countries_fatalities_df = countries_fatalities_df.round({'Fatality rate': 2})

In [ ]:
countries_fatalities_df.sort_values(by='Confirmed', ascending=False, inplace=True)

In [ ]:
countries_fatalities_df

**Belgium** has the highest fatality rate, while **Turkey** has the lowest one.

The fatality rate is effected by both factors:
- The quality of health care.
- The average age of the population.

In [ ]:
fig = make_subplots(rows=1, cols=3, shared_yaxes=True)

fig.add_trace(
    go.Bar(name='Confirmed', x=countries_fatalities_df['Confirmed'],
           y=countries_fatalities_df['Country'], orientation='h', marker_color='blue'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(name='Deaths', x=countries_fatalities_df['Deaths'],
           y=countries_fatalities_df['Country'], orientation='h', marker_color='red'),
    row=1, col=2
)

fig.add_trace(
    go.Bar(name='Fatality rate', x=countries_fatalities_df['Fatality rate'],
           y=countries_fatalities_df['Country'], orientation='h', marker_color='orange'),
    row=1, col=3
)

fig.update_layout(barmode='stack', autosize=False,
                  width=800, height=600,
                  title_text='<i><b>Countries fatality rate</b></i>',
                  yaxis_title="Country",
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  paper_bgcolor= 'rgba(0, 0, 0, 0)',
                  font={
                      'family': "Courier New, monospace",
                      'size': 14,
                      'color': "#eaeaea"
                  }
                 )

fig.show()

In [ ]:
save_fig_as_div(fig_obj=fig, file_name='most_affected_countries_fatality_rate_bar_chart.html')

## Cases per million people:

In [ ]:
cases_statistics_df = most_affected_df.merge(world_pop_df, left_on='Country',
                                             right_on='Country', how='inner')

In [ ]:
cases_statistics_df.head()

In [ ]:
cases_statistics_df['active cases per million'] = (1000000.0 * cases_statistics_df['Active']) / cases_statistics_df['population_size']
cases_statistics_df['deaths per million'] = (1000000.0 * cases_statistics_df['Deaths']) / cases_statistics_df['population_size']

In [ ]:
cases_statistics_df = cases_statistics_df.round({
    'active cases per million': 0,
    'deaths per million': 0
})

### Deaths per million people

In [ ]:
cases_statistics_df.sort_values(by='deaths per million', ascending=False, inplace=True)

fig = px.bar(data_frame=cases_statistics_df, x='Country', y='deaths per million',
             color='deaths per million', color_continuous_scale=px.colors.sequential.Reds)

fig.update_layout(autosize=False,
                  width=800, height=600,
                  title_text='<i><b>Deaths per million people</b></i>',
                  xaxis_title="Country", yaxis_title="Deaths per million",
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  paper_bgcolor= 'rgba(0, 0, 0, 0)',
                  font={
                      'family': "Courier New, monospace",
                      'size': 14,
                      'color': "#eaeaea"
                  }
                 )

fig.show()

In [ ]:
save_fig_as_div(fig_obj=fig, file_name='deaths_per_million_people_bar_chart.html')

### Active cases per million people

In [ ]:
cases_statistics_df.sort_values(by='active cases per million', inplace=True, ascending=False)

fig = px.bar(data_frame=cases_statistics_df, x='Country', y='active cases per million',
             color='active cases per million', color_continuous_scale=px.colors.sequential.YlOrBr)

fig.update_layout(autosize=False,
                  width=800, height=600,
                  title_text='<i><b>Active cases per million people</b></i>',
                  xaxis_title="Country", yaxis_title="Active cases per million",
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  paper_bgcolor= 'rgba(0, 0, 0, 0)',
                  font={
                      'family': "Courier New, monospace",
                      'size': 14,
                      'color': "#eaeaea"
                  }
                 )

# rotate x-axis ticks
fig.update_xaxes(tickangle=45)

fig.show()

In [ ]:
save_fig_as_div(fig_obj=fig, file_name='active_cases_per_million_people_bar_chart.html')

## Aging vs fatality-rate relation

In [ ]:
most_n = 40

In [ ]:
most_affected_countries_df = get_most_n_affected_countries(aggregated_df, most_n)

countries_fatalities_df = get_countries_fatalities(most_affected_countries_df)

In [ ]:
most_affected_countries_df.head()

In [ ]:
countries_fatalities_df.head()

In [ ]:
countries_ages_groups_df = countries_ages_groups_df.round({
    '0 to 14 years': 1,
    '15 to 64 years': 1,
    'over 65 years': 1
})

In [ ]:
countries_ages_groups_df.head()

In [ ]:
age_fatality_correlation_df = countries_fatalities_df.merge(countries_ages_groups_df,
                                                            left_on='Country',
                                                            right_on='Country', how='inner')

In [ ]:
age_fatality_correlation_df.head()

In [ ]:
fig = px.scatter(data_frame=age_fatality_correlation_df, x='over 65 years', y='Fatality rate',
                 color='Deaths',
                 size='Deaths', hover_data=['Country', 'Confirmed'],
                 color_continuous_scale=['orange', 'red'])

fig.update_layout(autosize=False,
                  width=1000, height=600,
                  title_text='<i><b>The correlation between fatality rate and aging</b></i>',
                  xaxis_title="Percentage of elderly (age over 65 years)",
                  yaxis_title="Fatality rate",
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  paper_bgcolor= 'rgba(0, 0, 0, 0)',
                  font={
                      'family': "Courier New, monospace",
                      'size': 14,
                      'color': "#eaeaea"
                  }
                 )

fig.show()

In [ ]:
save_fig_as_div(fig_obj=fig, file_name='age_vs_fatality_rate_scatter_plot.html')